In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import pyarrow
import fastparquet
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

pd.set_option('display.max_colwidth', None)

In [2]:
recipes = pd.read_parquet('../dataset/recipes-formatted.parquet.gzip')

In [12]:
recipes.vote_count.min()

15

# tf idf

In [3]:
df = pd.read_parquet('../dataset/recipes-formatted.parquet.gzip')
df = df[df['description'].notna()]
df.reset_index(inplace=True)

In [4]:
def process_text(text):
    # replace multiple spaces with one
    text = ' '.join(text.split())
    # lowercase
    text = text.lower()
 
    return text

df['description'] = df.apply(lambda x: process_text(x.description),axis=1)

In [5]:
tf_idf = TfidfVectorizer(stop_words='english')
 
tf_idf_matrix = tf_idf.fit_transform(df['description']);

In [6]:
cosine_similarity_matrix = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

In [7]:
def index_from_title(df,title):
    return df[df['recipe_title']==title].index.values[0]


# function that returns the title of the movie from its index
def title_from_index(df,index):
    return df[df.index==index].recipe_title.values[0]


# generating recommendations for given title
def recommendations(recipe_title, df,cosine_similarity_matrix,number_of_recommendations):
    index = index_from_title(df,recipe_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = [t[0] for t in similarity_scores_sorted[1:(number_of_recommendations+1)]]

    return df['recipe_title'].iloc[recommendations_indices]

In [8]:
dfkaki = pd.read_parquet('../dataset/recipes-formatted.parquet.gzip')

recommendations('Goan Sorak Curry Recipe', dfkaki, cosine_similarity_matrix, 13)

151                                                                  Goan Vegetable Curry Recipe
42                                          Goan Kaju Curry Recipe (Spicy Goan Cashew Nut Curry)
73                                                Goan Moolyacho Ross Recipe (Goan Radish Curry)
6715               केरला स्टाइल बैंगन करी रेसिपी - Kerala Style Eggplant Curry (Recipe In Hindi)
5933                                                                    Black Forest Cake Recipe
3176    Gujarati Badshahi Pulao Recipe - A Rich Preparation Of Rice, Vegetables, Nuts And Spices
2627                                                                     Pyaaz Ki Kachori Recipe
6176                                                          Homemade Whole Wheat Crepes Recipe
4809                               Aloo & Nariyal Ki Kachori Recipe (Pan Fried Farali Aloo Vada)
3372                                         Arbi Ke Kebab Recipe (Spicy Colocasia Kebab) Recipe
7904                      Andh

In [ ]:
df

# vectorize

In [49]:
recipes = pd.read_parquet('../dataset/recipes-formatted.parquet.gzip')

In [50]:
df = recipes

In [51]:
def return_values(value):
    values = []
    if value is not None:
        values.append(value.lower().replace(" ",""))

    return ' '.join(values)

def return_list_values(value):
    values = []
    if value is not None:
        for item in value:
            values.append(item.lower().replace(" ",""))

    return ' '.join(values)

df['record_health']=df.apply(lambda x: return_values(x.record_health),axis=1)
df['cuisine']=df.apply(lambda x: return_values(x.cuisine),axis=1)
df['course']=df.apply(lambda x: return_values(x.course),axis=1)
df['diet']=df.apply(lambda x: return_values(x.diet),axis=1)
df['prep_time']=df.apply(lambda x: return_values(x.prep_time),axis=1)
df['cook_time']=df.apply(lambda x: return_values(x.cook_time),axis=1)
df['category']=df.apply(lambda x: return_values(x.category),axis=1)
df['tags']=df.apply(lambda x: return_list_values(x.tags),axis=1)


In [53]:
w_record_health = 2
w_course = 1
w_cuisine = 5
w_diet = 3
w_prep_time = 6
w_cook_time = 5
w_category = 1
w_tags = 3
# function for merging features
 
def concatenate_features(df_row):
    return ' '.join([df_row['record_health']]*w_record_health) +' ' + \
            ' '.join([df_row['cuisine']]*w_cuisine)+' '+ \
            ' '.join([df_row['course']]*w_course)+' '+ \
            ' '.join([df_row['diet']]*w_diet)+' '+ \
            ' '.join([df_row['prep_time']]*w_prep_time)+' '+ \
            ' '.join([df_row['cook_time']]*w_cook_time)+' '+ \
            ' '.join([df_row['category']]*w_category)+' '+ \
            ' '.join([df_row['tags']]*w_tags)


In [54]:
df['features'] = df.apply(concatenate_features,axis=1)

In [55]:
# pre-processing text of features

def process_text(text):
    # replace multiple spaces with one
    text = ' '.join(text.split())
    # lowercase
    text=text.lower()

    return text

df['features'] = df.apply(lambda x: process_text(x.features),axis=1)


In [56]:
vect = CountVectorizer(stop_words='english')

vect_matrix = vect.fit_transform(df['features'])

cosine_similarity_matrix_count_based = cosine_similarity(vect_matrix, vect_matrix)

In [57]:
recommendations('Goan Sorak Curry Recipe', dfkaki, cosine_similarity_matrix_count_based, 13)

2500                           Goan Batata Recheado Recipe With Carrots & Beans
636                                                 Goan Prawns Vindaloo Recipe
2479                                                    Goan Gawar Bhaji Recipe
151                                                 Goan Vegetable Curry Recipe
6743           गोअन एग ड्राप करी रेसिपी - Goan Egg Drop Curry (Recipe In Hindi)
458                    Mushroom Caldine Recipe | Vegetarian Goan Mushroom Curry
5576            Doodanche Fov Recipe (Goan Style Milk Beaten Rice/Poha In Milk)
6537                     दही वाली भिन्डी रेसिपी - Dahi Bhindi (Recipe In Hindi)
6235                                                       Prawn Balchao Recipe
6279    The Goan Kelyachyo Fodi Recipe (Spicy & Crispy Pan Fried Banana Recipe)
5376                         Goan Style Dhayanche Fov Recipe (Curd Poha Recipe)
6653     Goan Style Sando Recipe (Steamed Rice Cakes Filled With Sweet Coconut)
7943                                    